In [120]:
%matplotlib inline
import random
import time 
import torch
import torch.nn as nn
from torch.nn import init
import numpy as np
import pandas as pd
from IPython import display
from collections import OrderedDict
import sys
import d2lzh as d2l
from matplotlib import pyplot as plt
from mpl_toolkits import mplot3d


## 4.1 模型构造


### 4.1.1 继承 MODULE 类来构造模型

Module 类是 nn 模块⾥提供的⼀个模型构造类，是所有神经⽹络模块的基类，我们可以继承它来定义
我们想要的模型。下⾯继承 Module 类构造本节开头提到的多层感知机。这⾥定义的 MLP 类᯿载了
Module 类的 __init__ 函数和 forward 函数。它们分别⽤于创建模型参数和定义前向计算。前向计
算也即正向传播。

In [46]:
import torch
from torch import nn

#  MLP`类中无须定义反向传播函数。
#  系统将通过自动求梯度而自动生成反向传播所需的`backward`函数。

class MLP(nn.Module):
    # 声明带有模型参数的层，这里声明了两个全连接层
    def __init__(self, **kwargs):
        # 调用MLP父类Module的构造函数来进行必要的初始化。这样在构造实例时还可以指定其他函数
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Linear(784, 256) # 隐藏层
        self.act = nn.ReLU()
        self.output = nn.Linear(256, 10)  # 输出层
        
    # 定义模型的前向计算，即如何根据输入x计算返回所需要的模型输出
    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)

In [53]:
# 实例化MLP类得到模型变量net
X = torch.rand(2,784)
net = MLP()
print(net)
net(X).shape # 2 10
net(X)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.0381,  0.3193,  0.2054,  0.0360, -0.0974, -0.0298, -0.0461,  0.0044,
          0.0503, -0.1548],
        [ 0.0811,  0.2190,  0.1913,  0.0872, -0.0776,  0.0412, -0.2329,  0.0557,
          0.0362, -0.2745]], grad_fn=<AddmmBackward>)

注意，这里并没有将`Module`类命名为`Layer`（层）或者`Model`（模型）之类的名字，这是因为该类是一个可供自由组建的部件。它的子类既可以是一个层（如PyTorch提供的`Linear`类），又可以是一个模型（如这里定义的`MLP`类），或者是模型的一个部分。我们下面通过两个例子来展示它的灵活性。

## 4.1.2 `Module`的子类
PyTorch还实现了继承自`Module`的可以方便构建模型的类: 如`Sequential`、`ModuleList`和`ModuleDict`等等。

### 4.1.2.1 `Sequential`类
当模型的前向计算为简单串联各个层的计算时，`Sequential`类可以通过更加简单的方式定义模型。这正是`Sequential`类的目的：它可以接收一个子模块的有序字典（OrderedDict）或者一系列子模块作为参数来逐一添加`Module`的实例，而模型的前向计算就是将这些实例按添加的顺序逐一计算。

In [62]:
# 实现一个与`Sequential`类有相同功能的`MySequential`类

class MySequential(nn.Module):
    def __init__(self, *args):
        super(MySequential, self).__init__()
        if len(args) == 1 and isinstance(args[0], OrderedDict): # 如果传入的是一个OrderedDict
            for key, module in args[0].items():
                self.add_module(key, module)  # add_module方法会将module添加进self._modules(一个OrderedDict)
        else:  # 传入的是一些Module
            for idx, module in enumerate(args):
                self.add_module(str(idx), module)
    def forward(self, input):
        # self._modules返回一个 OrderedDict，保证会按照成员添加时的顺序遍历成员
        for module in self._modules.values():
            input = module(input)
        return input

# 用`MySequential`类来实现前面描述的`MLP`类，
# 并使用随机初始化的模型做一次前向计算。
X = torch.rand(2,784)
net = MySequential(
        nn.Linear(784, 256),
        nn.ReLU(),
        nn.Linear(256, 10),
        )
print(net)
net(X)

# 可以观察到这里`MySequential`类的使用跟3.10节（多层感知机的简洁实现）中`Sequential`类的使用没什么区别。

MySequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.1853,  0.1869, -0.0744,  0.0117, -0.2149, -0.1122,  0.0158, -0.0172,
         -0.1010, -0.1256],
        [ 0.2070,  0.1760, -0.0286, -0.0443, -0.1740, -0.1356,  0.1430,  0.0134,
         -0.0137, -0.1042]], grad_fn=<AddmmBackward>)

#### 4.1.2.2 `ModuleList`类


In [63]:
# `ModuleList`接收一个子模块的列表作为输入，然后也可以类似List那样进行append和extend操作:

net = nn.ModuleList([nn.Linear(784, 256), nn.ReLU()])
net.append(nn.Linear(256, 10)) # # 类似List的append操作
print(net[-1])  # 类似List的索引访问
print(net)
# net(torch.zeros(1, 784)) # 会报NotImplementedError

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


In [65]:
# 4.1.2.3 ModuleDict 类
# ModuleDict 接收⼀个⼦模块的字典作为输⼊, 然后也可以类似字典那样进⾏添加访问操作:

net = nn.ModuleDict({
    'linear': nn.Linear(784, 256),
    'act': nn.ReLU(),
})
net['output'] = nn.Linear(256, 10) # 添加
print(net['linear']) # 访问
print(net.output)
print(net)

Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


In [71]:
# 4.1.3 构造复杂的模型
# 通过 get_constant 函数创建训练中不被迭代的参数，即常数参数

class FancyMLP(nn.Module):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)

        # 注意参数权重rand_weight（不是可训练模型参数
        self.rand_weight = torch.rand((20,20), requires_grad=True)
        self.linear = nn.Linear(20,20)

    def forward(self,x):
        x = self.linear(x)
        # 使⽤创建的常数参数，以及nn.functional中的relu函数和mm函数
        x = nn.functional.relu(torch.mm(x, self.rand_weight.data) + 1)

        # 复用全连接层，等价于两个全连接层共享参数
        x = self.linear(x)
        # 控制流，这里我们需要调用item函数来返回标量进行比较
        while x.norm().item() > 1:
            x /= 2
        if x.norm().item() < 0.8:
            x *= 10
        return x.sum()

X = torch.rand(2, 20)
net = FancyMLP()
print(net)
net(X)

FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


tensor(-0.6186, grad_fn=<SumBackward0>)

In [74]:
# 因为 FancyMLP 和 Sequential 类都是 Module 类的⼦类，所以我们可以嵌套调⽤它们

class NestMLP(nn.Module):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        self.net = nn.Sequential(nn.Linear(40, 30), nn.ReLU())
    def forward(self, x):
        return self.net(x)

net = nn.Sequential(NestMLP(), nn.Linear(30, 20), FancyMLP())

X = torch.rand(2, 40)
print(net)
net(X)

Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=40, out_features=30, bias=True)
      (1): ReLU()
    )
  )
  (1): Linear(in_features=30, out_features=20, bias=True)
  (2): FancyMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)


tensor(16.8084, grad_fn=<SumBackward0>)

### 小结

* 可以通过继承`Module`类来构造模型。
* `Sequential`、`ModuleList`、`ModuleDict`类都继承自`Module`类。
* 与`Sequential`不同，`ModuleList`和`ModuleDict`并没有定义一个完整的网络，它们只是将不同的模块存放在一起，需要自己定义`forward`函数。
* 虽然`Sequential`等类可以使模型构造更加简单，但直接继承`Module`类可以极大地拓展模型构造的灵活性。

## 4.2 模型参数的访问、初始化和共享



In [75]:
print(type(net.named_parameters()))
for name, param in net.named_parameters():
    print(name, param.size())
    

<class 'generator'>
0.net.0.weight torch.Size([30, 40])
0.net.0.bias torch.Size([30])
1.weight torch.Size([20, 30])
1.bias torch.Size([20])
2.linear.weight torch.Size([20, 20])
2.linear.bias torch.Size([20])


可⻅返回的名字⾃动加上了层数的索引作为前缀。 我们再来访问 net 中单层的参数。对于使
⽤ Sequential 类构造的神经⽹络，我们可以通过⽅括号 [] 来访问⽹络的任⼀层。索引0表示隐藏层
为 Sequential 实例最先添加的层

In [76]:
for name, param in net[0].named_parameters():
    print(name,param.size(),type(param))
# 返回的 param 的 类 型为 torch.nn.parameter.Parameter ，
# 其实这是 Tensor 的⼦类，
# 和 Tensor 不同的是如果⼀个 Tensor 是 Parameter ，那么它会⾃动被添加到模型的参数列表⾥

net.0.weight torch.Size([30, 40]) <class 'torch.nn.parameter.Parameter'>
net.0.bias torch.Size([30]) <class 'torch.nn.parameter.Parameter'>


In [77]:
class MyModel(nn.Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.weight1 = nn.Parameter(torch.rand(20, 20))
        self.weight2 = torch.rand(20, 20)
    def forward(self, x):
        pass

n = MyModel()
for name, param in n.named_parameters():
    print(name)

# 上⾯的代码中 weight1 在参数列表中但是 weight2 却没在参数列表中。

weight1


In [79]:
# 因为 Parameter 是 Tensor ，即 Tensor 拥有的属性它都有，⽐如可以根据 data 来访问参数数值，⽤
#  grad 来访问参数梯度。

net = nn.Sequential(nn.Linear(4, 3), nn.ReLU(), nn.Linear(3, 1)) #pytorch已进⾏默认初始化
print(net)
X = torch.rand(2, 4)
Y = net(X).sum()

weight_0 = list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad) # 反向传播前梯度为None
Y.backward()
print(weight_0.grad)

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)
tensor([[ 0.2485,  0.0379,  0.4240,  0.4590],
        [-0.0016,  0.2447,  0.1557,  0.0214],
        [ 0.1231,  0.3365,  0.2719,  0.4871]])
None
tensor([[ 0.1329,  0.2923,  0.3430,  0.4098],
        [-0.2323, -0.5109, -0.5995, -0.7163],
        [-0.2829, -0.6222, -0.7302, -0.8724]])


### 4.2.2 初始化模型参数
PyTorch的 init 模块⾥提供了多种预设的初始化⽅法

In [82]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean=0, std=0.01)
        print(name, param.data)

for name, param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param, val=0)
        print(name, param.data)

0.weight tensor([[ 0.0036, -0.0054,  0.0016, -0.0094],
        [ 0.0085, -0.0072, -0.0040, -0.0098],
        [ 0.0102, -0.0034,  0.0138,  0.0017]])
2.weight tensor([[ 0.0082,  0.0095, -0.0060]])
0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


### 4.2.3 ⾃定义初始化⽅法

先来看看PyTorch是怎么实现这些初始化⽅法的，例
如 torch.nn.init.normal_ ：
``` python
def normal_(tensor, mean=0, std=1):
    with torch.no_grad():
        return tensor.normal_(mean, std)
```
可以看到这就是一个inplace改变`Tensor`值的函数，而且这个过程是不记录梯度的。
类似的我们来实现一个自定义的初始化方法。在下面的例子里，我们令权重有一半概率初始化为0，有另一半概率初始化为$[-10,-5]$和$[5,10]$两个区间里均匀分布的随机数。

In [84]:
def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10,10)
        tensor *= (tensor.abs() >= 5).float()

for name,param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name,param.data)

0.weight tensor([[ 0.0000,  0.0000, -0.0000,  0.0000],
        [-8.4156, -8.7313, -9.4846, -7.4093],
        [-0.0000,  8.5241,  7.9221,  0.0000]])
2.weight tensor([[-7.8491,  0.0000, -0.0000]])


In [85]:
# 此外，参考2.3.2节，我们还可以通过改变这些参数的`data`来改写模型参数值同时不会影响梯度:

for name, param in net.named_parameters():
    if 'bias' in name:
        param.data += 1
        print(name, param.data)

0.bias tensor([1., 1., 1.])
2.bias tensor([1.])


### 4.2.4 共享模型参数

In [88]:
linear = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear, linear)
print(net)
for name, param in net.named_parameters():
    init.constant_(param, val=3)
    print(name, param.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


In [87]:
# 在内存中，这两个线性层其实⼀个对象

print(id(net[0]) == id(net[1]))
print(id(net[0].weight) == id(net[1].weight))

True
True


In [89]:
# 因为模型参数⾥包含了梯度，所以在反向传播计算时，这些共享的参数的梯度是累加的

x = torch.ones(1, 1)
y = net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad) # 单次梯度是3，两次所以就是6

tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])


## 4.4 自定义层


### 4.4.1 不含模型参数的⾃定义层

事实上，这和4.1节（模型构造）中介绍的使用`Module`类构造模型类似。

下面的`CenteredLayer`类通过继承`Module`类自定义了一个将输入减掉均值后输出的层，并将层的计算定义在了`forward`函数里。这个层里不含模型参数。

In [95]:
class CenteredLayer(nn.Module):
    def __init__(self, **kwargs):
        super(CenteredLayer, self).__init__(**kwargs)
    def forward(self, x):
        return x - x.mean()

layer = CenteredLayer()
layer(torch.tensor([1, 2, 3, 4, 5], dtype=torch.float))

net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())
y = net(torch.rand(4, 8))
y.mean().item()
# 因为均值是浮点数，所以它的值是⼀个很接近0的数。

-2.3283064365386963e-09

### 4.4.2 含模型参数的⾃定义层

在4.2节（模型参数的访问、初始化和共享）中介绍了 Parameter 类其实是 Tensor 的⼦类，如果⼀
个 Tensor 是 Parameter ，那么它会⾃动被添加到模型的参数列表⾥。所以在⾃定义含模型参数的层
时，我们应该将参数定义成 Parameter ，除了像4.2.1节那样直接定义成 Parameter 类外，还可以使
⽤ ParameterList 和 ParameterDict 分别定义参数的列表和字典。

ParameterList 接收⼀个 Parameter 实例的列表作为输⼊然后得到⼀个参数列表，使⽤的时候可以
⽤索引来访问某个参数，另外也可以使⽤ append 和 extend 在列表后⾯新增参数

In [108]:
class MyListDense(nn.Module):
    def __init__(self):
        super(MyListDense, self).__init__()
        self.params = nn.ParameterList([nn.Parameter(torch.randn(4, 4)) for i in range(3)])
        self.params.append(nn.Parameter(torch.randn(4, 1)))

    def forward(self, x):
        for i in range(len(self.params)):
            x = torch.mm(x, self.params[i])
        return x
net = MyListDense()
print(net)

MyListDense(
  (params): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 4x4]
      (1): Parameter containing: [torch.FloatTensor of size 4x4]
      (2): Parameter containing: [torch.FloatTensor of size 4x4]
      (3): Parameter containing: [torch.FloatTensor of size 4x1]
  )
)


⽽ ParameterDict 接收⼀个 Parameter 实例的字典作为输⼊然后得到⼀个参数字典，然后可以按照
字典的规则使⽤了。例如使⽤ update() 新增参数，使⽤ keys() 返回所有键值，使⽤ items() 返回
所有键值对等等

In [109]:
class MyDictDense(nn.Module):
    def __init__(self):
        super(MyDictDense, self).__init__()
        self.params = nn.ParameterDict({
                'linear1': nn.Parameter(torch.randn(4, 4)),
                'linear2': nn.Parameter(torch.randn(4, 1))
        })
        self.params.update({'linear3': nn.Parameter(torch.randn(4, 2))}) # 新增

    def forward(self, x, choice='linear1'):
        return torch.mm(x, self.params[choice])

net = MyDictDense()
print(net)

MyDictDense(
  (params): ParameterDict(
      (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
      (linear2): Parameter containing: [torch.FloatTensor of size 4x1]
      (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
  )
)


In [110]:
# 这样就可以根据传⼊的键值来进⾏不同的前向传播：
x = torch.ones(1, 4)

print(net(x, 'linear1'))
print(net(x, 'linear2'))
print(net(x, 'linear3'))


tensor([[-3.3331,  1.1265, -1.1502,  1.1859]], grad_fn=<MmBackward>)
tensor([[-0.1026]], grad_fn=<MmBackward>)
tensor([[-2.1482, -4.6740]], grad_fn=<MmBackward>)


In [111]:
# 也可以使⽤⾃定义层构造模型

net = nn.Sequential(
    MyDictDense(),
    MyListDense(),
)
print(net)
print(net(x))

Sequential(
  (0): MyDictDense(
    (params): ParameterDict(
        (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
        (linear2): Parameter containing: [torch.FloatTensor of size 4x1]
        (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
    )
  )
  (1): MyListDense(
    (params): ParameterList(
        (0): Parameter containing: [torch.FloatTensor of size 4x4]
        (1): Parameter containing: [torch.FloatTensor of size 4x4]
        (2): Parameter containing: [torch.FloatTensor of size 4x4]
        (3): Parameter containing: [torch.FloatTensor of size 4x1]
    )
  )
)
tensor([[-11.4312]], grad_fn=<MmBackward>)


## 4.5 读取和存储


在实际中，我们
有时需要把训练好的模型部署到很多不同的设备。在这种情况下，我们可以把内存中训练好的模型参数
存储在硬盘上供后续读取使⽤

In [114]:
# 4.5.1 读写 TENSOR

x = torch.ones(3)
y = torch.zeros(4)
torch.save([x, y], 'xy.pt')
xy_list = torch.load('xy.pt')
xy_list

torch.save({'x': x, 'y': y}, 'xy_dict.pt')
xy = torch.load('xy_dict.pt')
xy

{'x': tensor([1., 1., 1.]), 'y': tensor([0., 0., 0., 0.])}

### 4.5.2 读写模型
在 PyTorch 中 ， Module 的 可 学 习 参 数 ( 即权᯿和偏差 ) ，模块模型包含在参数中 ( 通 过
model.parameters() 访问)。 state_dict 是⼀个从参数名称隐射到参数 Tesnor 的字典对象。


In [115]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.hidden = nn.Linear(3, 2)
        self.act = nn.ReLU()
        self.output = nn.Linear(2, 1)
    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)
        
net = MLP()
net.state_dict()

OrderedDict([('hidden.weight',
              tensor([[-0.5658, -0.2694, -0.3134],
                      [-0.5666,  0.3352,  0.0934]])),
             ('hidden.bias', tensor([-0.0631,  0.0924])),
             ('output.weight', tensor([[ 0.1711, -0.1740]])),
             ('output.bias', tensor([-0.5386]))])

注意，只有具有可学习参数的层(卷积层、线性层等)才有 state_dict 中的条⽬。

优化器( optim )也有⼀个 state_dict ，其中包含关于优化器状态以及所使⽤的超参数的信息。

In [116]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.001,
   'momentum': 0.9,
   'dampening': 0,
   'weight_decay': 0,
   'nesterov': False,
   'params': [0, 1, 2, 3]}]}

#### 4.5.2.2 保存和加载模型

PyTorch中保存和加载训练模型有两种常见的方法:
1. 仅保存和加载模型参数(`state_dict`)；
2. 保存和加载整个模型。
3. 
##### 1. 保存和加载`state_dict`(推荐方式)

保存：
``` python
torch.save(model.state_dict(), PATH) # 推荐的文件后缀名是pt或pth
```

加载：
``` python
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
```

##### 2. 保存和加载整个模型
保存：
``` python
torch.save(model, PATH)
```
加载：
``` python
model = torch.load(PATH)
```

我们采用推荐的方法一来实验一下:
``` python
X = torch.randn(2, 3)
Y = net(X)

PATH = "./net.pt"
torch.save(net.state_dict(), PATH)

net2 = MLP()
net2.load_state_dict(torch.load(PATH))
Y2 = net2(X)
Y2 == Y
```
输出：
```
tensor([[1],
        [1]], dtype=torch.uint8)
```

因为这`net`和`net2`都有同样的模型参数，那么对同一个输入`X`的计算结果将会是一样的。上面的输出也验证了这一点。

此外，还有一些其他使用场景，例如GPU与CPU之间的模型保存与读取、使用多块GPU的模型的存储等等，使用的时候可以参考[官方文档](https://pytorch.org/tutorials/beginner/saving_loading_models.html)。

## 小结

* 通过`save`函数和`load`函数可以很方便地读写`Tensor`。
* 通过`save`函数和`load_state_dict`函数可以很方便地读写模型的参数。